In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import paired_cosine_distances, paired_euclidean_distances, paired_manhattan_distances
from scipy.stats import pearsonr, spearmanr
from sklearn import metrics
import torch
import time
import umap
import umap.plot
import time
import itertools
# from line_profiler import LineProfiler

from utils import prepare_dataset, calculate_all_metric, find_spearman_pearson

import logging
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.WARNING,  # set 3rd party logs to warning (for hiding it)
)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# profile = LineProfiler()

#### Input

In [3]:
df_hscode4 = prepare_dataset()
df_hscode4

,hscode4_text,HSCode2
0,ม้า ลา และล่อมีชีวิต,01
1,สัตว์จำพวกโคกระบือ มีชีวิต,01
2,สุกรมีชีวิต,01
3,แกะและแพะมีชีวิต,01
4,สัตว์ปีกเลี้ยงมีชีวิต ได้แก่ ไก่ชนิดแกลลัสโดเม...,01
...,...,...
2422,"Original engravings, prints and lithographs. ...",97
2423,"Original sculptures and statuary, in any mater...",97
2424,"Postage or revenue stamps, stamp-postmarks, fi...",97
2425,Collections and collectors' pieces of zoologic...,97


In [17]:
# from googletrans import Translator
# translator = Translator()
# # translator.translate('안녕하세요.', dest='ja')

translator.translate("ม้า ลา และล่อมีชีวิต", src='auto', dest='en').text

'Live horses, donkeys and mules'

In [14]:
# ! pip install googletrans==3.1.0a0

In [19]:
import googletrans
from googletrans import Translator
import pandas as pd

translator = Translator()
df_hscode4['TranslatedText'] = df_hscode4['hscode4_text'].apply(lambda x: translator.translate(x, src='th', dest='en').text )

TypeError: 'NoneType' object is not iterable

In [35]:
test = list(df_hscode4['hscode4_text'].values)

In [36]:
translations = translator.translate(test, dest='en')

In [44]:
en_translations = np.array([translation.text for translation in translations])

In [46]:
# en_translations

In [47]:
df_hscode4['TranslatedText'] = en_translations

In [48]:
df_hscode4

,hscode4_text,HSCode2,TranslatedText
0,ม้า ลา และล่อมีชีวิต,01,"Live horses, donkeys and mules"
1,สัตว์จำพวกโคกระบือ มีชีวิต,01,Bovine animals live
2,สุกรมีชีวิต,01,live pig
3,แกะและแพะมีชีวิต,01,Live sheep and goats
4,สัตว์ปีกเลี้ยงมีชีวิต ได้แก่ ไก่ชนิดแกลลัสโดเม...,01,"Live poultry include gallus domesticus, ducks,..."
...,...,...,...
2422,"Original engravings, prints and lithographs. ...",97,"Original engravings, prints and lithographs."
2423,"Original sculptures and statuary, in any mater...",97,"Original sculptures and statuary, in any mater..."
2424,"Postage or revenue stamps, stamp-postmarks, fi...",97,"Postage or revenue stamps, stamp-postmarks, fi..."
2425,Collections and collectors' pieces of zoologic...,97,Collections and collectors' pieces of zoologic...


In [49]:
# tt = df_hscode4['hscode4_text'][:100].apply(lambda x: translator.translate(x, src='th', dest='en').text )

In [21]:
# df_hscode4

In [50]:
df_hscode4.to_csv('dataset/semantic_similarity/hscode4_en_translate.csv',index=False)

In [4]:
df_hscode4

,hscode4_text,HSCode2
0,ม้า ลา และล่อมีชีวิต,01
1,สัตว์จำพวกโคกระบือ มีชีวิต,01
2,สุกรมีชีวิต,01
3,แกะและแพะมีชีวิต,01
4,สัตว์ปีกเลี้ยงมีชีวิต ได้แก่ ไก่ชนิดแกลลัสโดเม...,01
...,...,...
2422,"Original engravings, prints and lithographs. ...",97
2423,"Original sculptures and statuary, in any mater...",97
2424,"Postage or revenue stamps, stamp-postmarks, fi...",97
2425,Collections and collectors' pieces of zoologic...,97


In [4]:
df_cross = df_hscode4.merge(df_hscode4, how='cross')
df_cross.loc[ df_cross['HSCode2_x'] == df_cross['HSCode2_y'], 'score'] = 1
df_cross.loc[ df_cross['HSCode2_x'] != df_cross['HSCode2_y'], 'score'] = 0
df_cross

,hscode4_text_x,HSCode2_x,hscode4_text_y,HSCode2_y,score
0,ม้า ลา และล่อมีชีวิต,01,ม้า ลา และล่อมีชีวิต,01,1.0
1,ม้า ลา และล่อมีชีวิต,01,สัตว์จำพวกโคกระบือ มีชีวิต,01,1.0
2,ม้า ลา และล่อมีชีวิต,01,สุกรมีชีวิต,01,1.0
3,ม้า ลา และล่อมีชีวิต,01,แกะและแพะมีชีวิต,01,1.0
4,ม้า ลา และล่อมีชีวิต,01,สัตว์ปีกเลี้ยงมีชีวิต ได้แก่ ไก่ชนิดแกลลัสโดเม...,01,1.0
...,...,...,...,...,...
5890324,Antiques of an age exceeding one hundred years...,97,"Original engravings, prints and lithographs. ...",97,1.0
5890325,Antiques of an age exceeding one hundred years...,97,"Original sculptures and statuary, in any mater...",97,1.0
5890326,Antiques of an age exceeding one hundred years...,97,"Postage or revenue stamps, stamp-postmarks, fi...",97,1.0
5890327,Antiques of an age exceeding one hundred years...,97,Collections and collectors' pieces of zoologic...,97,1.0


In [5]:
T = df_hscode4['hscode4_text'].values
Y = df_hscode4['HSCode2'].values
labels = df_cross['score'].values

In [8]:
labels.shape

(5890329,)

In [6]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [7]:
X = model.encode(T, convert_to_tensor=True)

In [8]:
X.shape

torch.Size([2427, 384])

In [9]:
# new_labels = labels[:2427]

In [11]:
# X.min()

In [10]:
# cos_sim

In [12]:
# df_cross['cosine_sim'] = cosine_scores

In [13]:
# df_cross[:60]

In [14]:
cos_sim = util.cos_sim(X, X)
dot_products = util.dot_score(X,X)

cosine_scores = torch.reshape(cos_sim, (labels.shape[0], ))
dot_products = torch.reshape(dot_products, (labels.shape[0], ))

In [15]:
i = 100
n = 100
a = cos_sim[i,:].numpy()
index = np.argsort(a)[::-1][:n]

In [16]:
df = df_hscode4.iloc[index]
df['similarity'] = a[index]
df

/tmp/ipykernel_195769/1074210744.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['similarity'] = a[index]


,hscode4_text,HSCode2,similarity
100,สตาร์ชและอินูลิน,11,1.000001
881,ผงนิกเกิลและเกล็ดนิกเกิล ...,75,0.709436
229,สินแร่และหัวแร่นิกเกิล ...,26,0.694377
293,ไดไทโอไนต์และซัลโฟซิเลต,28,0.690017
295,ซัลเฟต อะลัม เพอร์ออกโซซัลเฟต (เพอร์ซัลเฟต),28,0.685537
...,...,...,...
683,เชิ้ตของบุรุษหรือเด็กชาย ถักแบบนิตหรือแบบโครเชต์,61,0.431368
2,สุกรมีชีวิต,01,0.430594
84,ขิง หญ้าฝรั่น ขมิ้น ไทม์ ใบเบย์ หอมแขก และเครื...,09,0.428334
914,ท่อน เส้น โพรไฟล์ และลวดทำด้วยดีบุก,80,0.426035


In [4]:
df_hscode4['hscode4_text'] = df_hscode4['hscode4_text'].astype('string')
df_hscode4['HSCode2'] = df_hscode4['HSCode2'].astype('string')

In [5]:
df_hscode4.to_csv('dataset/semantic_similarity/hscode4.csv',index=False)

In [58]:
df_hscode4

,hscode4_text,HSCode2
0,ม้า ลา และล่อมีชีวิต,01
1,สัตว์จำพวกโคกระบือ มีชีวิต,01
2,สุกรมีชีวิต,01
3,แกะและแพะมีชีวิต,01
4,สัตว์ปีกเลี้ยงมีชีวิต ได้แก่ ไก่ชนิดแกลลัสโดเม...,01
...,...,...
2422,"Original engravings, prints and lithographs. ...",97
2423,"Original sculptures and statuary, in any mater...",97
2424,"Postage or revenue stamps, stamp-postmarks, fi...",97
2425,Collections and collectors' pieces of zoologic...,97


In [67]:
df_hscode4.iloc[2421]['hscode4_text']

''

In [39]:
cosine_scores.shape

torch.Size([5890329])

In [65]:
# cos_sim = util.cos_sim(X, X)
# dot_products = util.dot_score(X,X)

# cosine_scores = torch.reshape(cos_sim, (labels.shape[0], ))
# dot_products = torch.reshape(dot_products, (labels.shape[0], ))

In [7]:
parameters = {
    'distiluse-base-multilingual-cased-v2': {
    },
    'paraphrase-multilingual-MiniLM-L12-v2': {
    },
    'paraphrase-multilingual-mpnet-base-v2': {
    },
    
    
}

model_result = {
    'distiluse-base-multilingual-cased-v2': {},
    'paraphrase-multilingual-MiniLM-L12-v2': {},
    'paraphrase-multilingual-mpnet-base-v2': {},
}

In [13]:
# @profile
def run_model(model_name):
    start_time = time.time()
    model = SentenceTransformer(model_name)
    load_model_time = time.time() - start_time
    print('   load_model_time:', load_model_time)
    
    start_time = time.time()
    X = model.encode(T, convert_to_tensor=True)
    encode_time = time.time() - start_time
    print('   encode_time:', load_model_time)
    
    start_time = time.time()
    result = find_spearman_pearson(X, labels)
    spearman_pearson_time = time.time() - start_time
    
    result['load_model_time'] = load_model_time
    result['encode_time'] = encode_time
    result['spearman_pearson_time'] = spearman_pearson_time
    
    # with open(f'profiling/profiling_{model_name}.txt', 'w') as stream:
    #     profile.print_stats(stream=stream)
    
    return result 
    

In [14]:
for key, value in parameters.items():
    print(f'--- {key} ---')
    result = run_model(key)   
    model_result[key] = result 
    print(result)
        


--- distiluse-base-multilingual-cased-v2 ---
   load_model_time: 9.80741024017334
   encode_time: 9.80741024017334
{'spearman_cosine': 0.12185646569416776, 'pearson_cosine': 0.18877965377804343, 'spearman_dot': 0.1135390022756929, 'pearson_dot': 0.18198868809745544, 'load_model_time': 9.80741024017334, 'encode_time': 147.58377933502197, 'spearman_pearson_time': 7.422705173492432}
--- paraphrase-multilingual-MiniLM-L12-v2 ---
   load_model_time: 1.8617308139801025
   encode_time: 1.8617308139801025
{'spearman_cosine': 0.16805728795253272, 'pearson_cosine': 0.23396830402449054, 'spearman_dot': 0.1567375596275878, 'pearson_dot': 0.22078572014800174, 'load_model_time': 1.8617308139801025, 'encode_time': 66.21149754524231, 'spearman_pearson_time': 3.884265184402466}
--- paraphrase-multilingual-mpnet-base-v2 ---
   load_model_time: 6.890433073043823
   encode_time: 6.890433073043823
{'spearman_cosine': 0.17721252569385856, 'pearson_cosine': 0.2390359997085682, 'spearman_dot': 0.1691917671565

In [15]:
model_result_temp = model_result.copy()

In [16]:
model_result_temp

{'distiluse-base-multilingual-cased-v2': {'spearman_cosine': 0.12185646569416776,
  'pearson_cosine': 0.18877965377804343,
  'spearman_dot': 0.1135390022756929,
  'pearson_dot': 0.18198868809745544,
  'load_model_time': 9.80741024017334,
  'encode_time': 147.58377933502197,
  'spearman_pearson_time': 7.422705173492432},
 'paraphrase-multilingual-MiniLM-L12-v2': {'spearman_cosine': 0.16805728795253272,
  'pearson_cosine': 0.23396830402449054,
  'spearman_dot': 0.1567375596275878,
  'pearson_dot': 0.22078572014800174,
  'load_model_time': 1.8617308139801025,
  'encode_time': 66.21149754524231,
  'spearman_pearson_time': 3.884265184402466},
 'paraphrase-multilingual-mpnet-base-v2': {'spearman_cosine': 0.17721252569385856,
  'pearson_cosine': 0.2390359997085682,
  'spearman_dot': 0.16919176715652487,
  'pearson_dot': 0.23484528793003517,
  'load_model_time': 6.890433073043823,
  'encode_time': 207.694682598114,
  'spearman_pearson_time': 4.681907653808594}}

In [17]:
result_list = []
for key, value in model_result_temp.items():
    model_result_temp[key]['model_name'] = key
    result_list.append(model_result_temp[key])
    
df_result_all = pd.DataFrame(result_list)
df_result_all.to_csv('spearman_pearson_output.csv',index=False, sep='\t')

In [92]:
model_result

{'distiluse-base-multilingual-cased-v2': {'model_name': 'distiluse-base-multilingual-cased-v2'},
 'paraphrase-multilingual-MiniLM-L12-v2': {'model_name': 'paraphrase-multilingual-MiniLM-L12-v2'},
 'paraphrase-multilingual-mpnet-base-v2': {'model_name': 'paraphrase-multilingual-mpnet-base-v2'}}

In [85]:
# del model_result['model_name']

#### Umap

In [26]:
umap_params = {
    'n_components': np.arange(2, 11, 1),
    'n_neighbors': [2,4,8,16,32,64],
}

parameters = {
    'distiluse-base-multilingual-cased-v2': {
    },
    'paraphrase-multilingual-MiniLM-L12-v2': {
    },
    'paraphrase-multilingual-mpnet-base-v2': {
    },   
    
}
model_result_umap = {
    'distiluse-base-multilingual-cased-v2': [],
    'paraphrase-multilingual-MiniLM-L12-v2': [],
    'paraphrase-multilingual-mpnet-base-v2': [],
}

In [27]:
def run_model_umap(X, umap_param):
    start_time = time.time()
    X_umap = umap.UMAP(**umap_param).fit_transform(X)
    umap_time = time.time() - start_time
    print('   umap_time:', umap_time)
    
    start_time = time.time()
    result = find_spearman_pearson(X_umap, labels)
    spearman_pearson_time = time.time() - start_time
    
    result['umap_time'] = umap_time
    result['spearman_pearson_time'] = spearman_pearson_time
    
    return result 

In [28]:
umap_keys = list(umap_params.keys())
umap_values = umap_params.values()
umap_all_combinations = list(itertools.product(*umap_values))


for key, value in parameters.items():
    print(f'--- {key} ---')
    start_time = time.time()
    model = SentenceTransformer(key)
    load_model_time = time.time() - start_time
    print('   load_model_time:', load_model_time)
    
    start_time = time.time()
    X = model.encode(T, convert_to_tensor=True)
    encode_time = time.time() - start_time
    print('   encode_time:', encode_time)
    
    for umap_parameter_value in umap_all_combinations:
        umap_parameter_result = {}
        for i_parameter in range(len(umap_parameter_value)):
            umap_parameter_result[umap_keys[i_parameter]] = umap_parameter_value[i_parameter]
        print('   umap_parameter:', umap_parameter_result)
        result = run_model_umap(X, umap_param=umap_parameter_result) 
        result['load_model_time'] = load_model_time
        result['encode_time'] = encode_time  
        result['umap_param'] = umap_parameter_result
        result['model_name'] = key
        model_result_umap[key].append(result) 
        print(len(model_result_umap[key]))
        print(result) 

--- distiluse-base-multilingual-cased-v2 ---
   load_model_time: 0.8553686141967773
   encode_time: 175.56898069381714
   umap_parameter: {'n_components': 2, 'n_neighbors': 2}


/home/patcharapon/miniconda3/envs/mdm/lib/python3.8/site-packages/umap/spectral.py:260: UserWarning: WARNING: spectral initialisation failed! The eigenvector solver
failed. This is likely due to too small an eigengap. Consider
adding some noise or jitter to your data.

Falling back to random initialisation!
  warn(


   umap_time: 8.837761878967285
1
{'spearman_cosine': 0.035263988552731614, 'pearson_cosine': 0.02963233237752125, 'spearman_dot': 0.028810052038481485, 'pearson_dot': 0.03176504802515109, 'umap_time': 8.837761878967285, 'spearman_pearson_time': 3.3302886486053467, 'load_model_time': 0.8553686141967773, 'encode_time': 175.56898069381714, 'umap_param': {'n_components': 2, 'n_neighbors': 2}, 'model_name': 'distiluse-base-multilingual-cased-v2'}
   umap_parameter: {'n_components': 2, 'n_neighbors': 4}
   umap_time: 10.313378095626831
2
{'spearman_cosine': 0.10343262525275582, 'pearson_cosine': 0.060565530957452623, 'spearman_dot': 0.07297796644066232, 'pearson_dot': 0.07675776527408584, 'umap_time': 10.313378095626831, 'spearman_pearson_time': 4.079493761062622, 'load_model_time': 0.8553686141967773, 'encode_time': 175.56898069381714, 'umap_param': {'n_components': 2, 'n_neighbors': 4}, 'model_name': 'distiluse-base-multilingual-cased-v2'}
   umap_parameter: {'n_components': 2, 'n_neighbo

/home/patcharapon/miniconda3/envs/mdm/lib/python3.8/site-packages/umap/spectral.py:260: UserWarning: WARNING: spectral initialisation failed! The eigenvector solver
failed. This is likely due to too small an eigengap. Consider
adding some noise or jitter to your data.

Falling back to random initialisation!
  warn(


   umap_time: 7.336989402770996
13
{'spearman_cosine': 0.09351874191341936, 'pearson_cosine': 0.08102610482433499, 'spearman_dot': 0.07727211156169804, 'pearson_dot': 0.08453386210545469, 'umap_time': 7.336989402770996, 'spearman_pearson_time': 6.253692626953125, 'load_model_time': 0.8553686141967773, 'encode_time': 175.56898069381714, 'umap_param': {'n_components': 4, 'n_neighbors': 2}, 'model_name': 'distiluse-base-multilingual-cased-v2'}
   umap_parameter: {'n_components': 4, 'n_neighbors': 4}
   umap_time: 7.522204160690308
14
{'spearman_cosine': 0.11078789130719231, 'pearson_cosine': 0.03973038126114755, 'spearman_dot': 0.04529572584863124, 'pearson_dot': 0.058297128717003814, 'umap_time': 7.522204160690308, 'spearman_pearson_time': 2.9616570472717285, 'load_model_time': 0.8553686141967773, 'encode_time': 175.56898069381714, 'umap_param': {'n_components': 4, 'n_neighbors': 4}, 'model_name': 'distiluse-base-multilingual-cased-v2'}
   umap_parameter: {'n_components': 4, 'n_neighbors

...

   umap_time: 7.139737606048584
49
{'spearman_cosine': 0.07156192908630164, 'pearson_cosine': 0.07876755202289444, 'spearman_dot': 0.04964083583812503, 'pearson_dot': 0.06558763550208963, 'umap_time': 7.139737606048584, 'spearman_pearson_time': 2.778484582901001, 'load_model_time': 2.2524263858795166, 'encode_time': 52.06822109222412, 'umap_param': {'n_components': 10, 'n_neighbors': 2}, 'model_name': 'paraphrase-multilingual-MiniLM-L12-v2'}
   umap_parameter: {'n_components': 10, 'n_neighbors': 4}
   umap_time: 7.988241195678711
50
{'spearman_cosine': 0.14955561333970013, 'pearson_cosine': 0.08757985573960418, 'spearman_dot': -0.004869531327084721, 'pearson_dot': -0.001087120585404392, 'umap_time': 7.988241195678711, 'spearman_pearson_time': 3.0251851081848145, 'load_model_time': 2.2524263858795166, 'encode_time': 52.06822109222412, 'umap_param': {'n_components': 10, 'n_neighbors': 4}, 'model_name': 'paraphrase-multilingual-MiniLM-L12-v2'}
   umap_parameter: {'n_components': 10, 'n_ne

/home/patcharapon/miniconda3/envs/mdm/lib/python3.8/site-packages/umap/spectral.py:260: UserWarning: WARNING: spectral initialisation failed! The eigenvector solver
failed. This is likely due to too small an eigengap. Consider
adding some noise or jitter to your data.

Falling back to random initialisation!
  warn(


   umap_time: 8.649554967880249
1
{'spearman_cosine': 0.0436162056282192, 'pearson_cosine': 0.03641057543155208, 'spearman_dot': 0.029898817885833772, 'pearson_dot': 0.029779989316109663, 'umap_time': 8.649554967880249, 'spearman_pearson_time': 2.943350315093994, 'load_model_time': 2.2189950942993164, 'encode_time': 159.79424285888672, 'umap_param': {'n_components': 2, 'n_neighbors': 2}, 'model_name': 'paraphrase-multilingual-mpnet-base-v2'}
   umap_parameter: {'n_components': 2, 'n_neighbors': 4}
   umap_time: 8.88732385635376
2
{'spearman_cosine': 0.1510046882736703, 'pearson_cosine': 0.09768438557987945, 'spearman_dot': 0.08530429619555775, 'pearson_dot': 0.07844928747343155, 'umap_time': 8.88732385635376, 'spearman_pearson_time': 6.1964006423950195, 'load_model_time': 2.2189950942993164, 'encode_time': 159.79424285888672, 'umap_param': {'n_components': 2, 'n_neighbors': 4}, 'model_name': 'paraphrase-multilingual-mpnet-base-v2'}
   umap_parameter: {'n_components': 2, 'n_neighbors': 

/home/patcharapon/miniconda3/envs/mdm/lib/python3.8/site-packages/umap/spectral.py:260: UserWarning: WARNING: spectral initialisation failed! The eigenvector solver
failed. This is likely due to too small an eigengap. Consider
adding some noise or jitter to your data.

Falling back to random initialisation!
  warn(
/home/patcharapon/miniconda3/envs/mdm/lib/python3.8/site-packages/umap/spectral.py:260: UserWarning: WARNING: spectral initialisation failed! The eigenvector solver
failed. This is likely due to too small an eigengap. Consider
adding some noise or jitter to your data.

Falling back to random initialisation!
  warn(


   umap_time: 6.641707897186279
13
{'spearman_cosine': 0.08362263882448605, 'pearson_cosine': 0.07672951716670956, 'spearman_dot': 0.04089647370015042, 'pearson_dot': 0.04417498124388226, 'umap_time': 6.641707897186279, 'spearman_pearson_time': 5.4426960945129395, 'load_model_time': 2.2189950942993164, 'encode_time': 159.79424285888672, 'umap_param': {'n_components': 4, 'n_neighbors': 2}, 'model_name': 'paraphrase-multilingual-mpnet-base-v2'}
   umap_parameter: {'n_components': 4, 'n_neighbors': 4}
   umap_time: 7.97737455368042
14
{'spearman_cosine': 0.15809757524286766, 'pearson_cosine': 0.08873616788109137, 'spearman_dot': 0.03913234702193083, 'pearson_dot': 0.042900658464552516, 'umap_time': 7.97737455368042, 'spearman_pearson_time': 2.8122591972351074, 'load_model_time': 2.2189950942993164, 'encode_time': 159.79424285888672, 'umap_param': {'n_components': 4, 'n_neighbors': 4}, 'model_name': 'paraphrase-multilingual-mpnet-base-v2'}
   umap_parameter: {'n_components': 4, 'n_neighbor

In [29]:
model_result_umap

{'distiluse-base-multilingual-cased-v2': [{'spearman_cosine': 0.035263988552731614,
   'pearson_cosine': 0.02963233237752125,
   'spearman_dot': 0.028810052038481485,
   'pearson_dot': 0.03176504802515109,
   'umap_time': 8.837761878967285,
   'spearman_pearson_time': 3.3302886486053467,
   'load_model_time': 0.8553686141967773,
   'encode_time': 175.56898069381714,
   'umap_param': {'n_components': 2, 'n_neighbors': 2},
   'model_name': 'distiluse-base-multilingual-cased-v2'},
  {'spearman_cosine': 0.10343262525275582,
   'pearson_cosine': 0.060565530957452623,
   'spearman_dot': 0.07297796644066232,
   'pearson_dot': 0.07675776527408584,
   'umap_time': 10.313378095626831,
   'spearman_pearson_time': 4.079493761062622,
   'load_model_time': 0.8553686141967773,
   'encode_time': 175.56898069381714,
   'umap_param': {'n_components': 2, 'n_neighbors': 4},
   'model_name': 'distiluse-base-multilingual-cased-v2'},
  {'spearman_cosine': 0.10268349527541087,
   'pearson_cosine': 0.038383393

In [30]:
model_result_umap_temp = model_result_umap.copy()

In [31]:
result_list = []
for key, value in model_result_umap_temp.items():
    # model_result_umap_temp[key]['model_name'] = key
    result_list.extend(model_result_umap_temp[key])
    
df_result_all = pd.DataFrame(result_list)
df_result_all.to_csv('spearman_pearson_umap_output.csv',index=False, sep='\t')

In [ ]:
--- distiluse-base-multilingual-cased-v2 ---
   load_model_time: 10.39717721939087
   encode_time: 149.7373948097229

In [64]:
# eval_pearson_cosine, _ = pearsonr(labels, cosine_scores)
# eval_spearman_cosine, _ = spearmanr(labels, cosine_scores)

# eval_pearson_dot, _ = pearsonr(labels, dot_products)
# eval_spearman_dot, _ = spearmanr(labels, dot_products)

In [ ]:
# cosine_scores = 1 - (paired_cosine_distances(X, X))
# manhattan_distances = -paired_manhattan_distances(embeddings1, embeddings2)
# euclidean_distances = -paired_euclidean_distances(embeddings1, embeddings2)
# dot_products = [np.dot(emb1, emb2) for emb1, emb2 in zip(embeddings1, embeddings2)]


# eval_pearson_cosine, _ = pearsonr(labels, cosine_scores)
# eval_spearman_cosine, _ = spearmanr(labels, cosine_scores)

# eval_pearson_manhattan, _ = pearsonr(labels, manhattan_distances)
# eval_spearman_manhattan, _ = spearmanr(labels, manhattan_distances)

# eval_pearson_euclidean, _ = pearsonr(labels, euclidean_distances)
# eval_spearman_euclidean, _ = spearmanr(labels, euclidean_distances)

# eval_pearson_dot, _ = pearsonr(labels, dot_products)
# eval_spearman_dot, _ = spearmanr(labels, dot_products)